# Human Detection Project

En este proyecto vamos a desarrollar un detector de personas en imágenes.


## Índice del proyecto

- [Lectura de imágenes](#Lectura-de-imágenes)
    - [Fotos Train](#Fotos-Train)
    - [Fotos Test](#Fotos-Test)
- [Tratamiento de la imagenes Train](#Tratamiento-de-la-imagenes-Train)
    - [Recorte Fotos Train Negativa](#Recorte-Fotos-Train-Negativa)
    - [Ampliación Del Rango Dinámico, Suavizado y Normalización](#Ampliación-Del-Rango-Dinámico,-Suavizado-y-Normalización)
- [Extracción de Caracterísitcas](#Extracción-de-Caracterísitcas)
- [Entrenamiento del modelo](#Entrenamiento-del-modelo)
- [Supresión No Maximos](#Supresión-No-Maximos)
- [Prueba del Modelo](#Prueba-del-Modelo)


Importamos todas las librerías y funciones que vamos a utilizar en este proyecto.

In [1]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from sklearn import svm
from sklearn.decomposition import PCA
from skimage.transform import pyramid_gaussian
import matplotlib.pyplot as plt

La función para mostrar imágenes.

In [2]:
def showImg(im,text = 'xd'):
    cv2.imshow(text,im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Lectura de imágenes

La función lee desde el argumento de entrada todas las fotos del directorio y las guarda en un diccionario.

In [3]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

### Fotos Train 

Esta función crea imágenes invertidas para tener más ejemplos de Train.

In [4]:
def duplicaTrani(fotos):
    indx = len(fotos)
    for i in range(len(fotos)):
        fotos[indx+i] = cv2.flip(fotos[i], 1)
        
    return fotos

Leemos las imágenes Train positiva y Train negativa indicando el directorio donde se encuentran.
(las imágenes del Train negativo hay que recortarlas posteriormente)

In [5]:
path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain0 = leerFotos(path)
fotosPosTrain0 = duplicaTrani(fotosPosTrain0)

path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

### Fotos Test

Leemos las imágenes del Test.

In [6]:
path = 'C:\\Users\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\Test\\pos'
fotosTest0 = leerFotos(path)

## Tratamiento de la imagenes Train

### Recorte Fotos Train Negativa

Como el tamaño de las imágenes Train negativas no coincide con las Train positivas no podemos entrenarlas por HOG, por lo tanto de alguna forma hay que dejarlas con el mismo tamaño.

Lo que vamos a hacer es escoger aleatoriamente en cada imagen train negativa bloques del tamaño de las Train positivas.

Los parámetros de entrada son:
* **winSize**: el tamaño del bloque
* **dataNeg**: las fotos Train negativas sin tratar
* **numeroCortesxPh**: número de bloques que va a sacar por cada Train negativa

y nos devolve las fotos train negativas con el mismo tamaño que las positivas.


In [7]:
def generaImagenesRecortadas(winSize,dataNeg,numeroCortesxPh = 20):
    rowsFotoPersona = winSize[0]
    colsFotoPersona = winSize[1]
    
    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        rowsFotoEntorno = dataNeg[i].shape[0]
        colsFotoEntorno = dataNeg[i].shape[1]
        dimCol = colsFotoEntorno - colsFotoPersona
        dimRow = rowsFotoEntorno - rowsFotoPersona
        
        for j in range(numeroCortesxPh):    
            randRow = random.randint(0,dimRow-1)
            randCol = random.randint(0,dimCol-1)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona ,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroCortesxPh*i] = randomImage
            
    return fotosNegTrain

Antes de entrenar el conjunto Train vamos a hacer una serie de tratamientos a las imágenes para conseguir mejores resultados.

In [53]:
winSize = fotosPosTrain0[0].shape
fotosNegTrain0 = generaImagenesRecortadas(winSize,fotosNegTrainSinTratar)

### Ampliación Del Rango Dinámico, Suavizado y Normalización

Al ampliar el rango dinámico de las fotos aumentamos el contraste para un mejor extracción de características.

El suavizado nos permite eliminar ruidos y detalles innecesarios.

In [54]:
def ampRangoSet(fotos):
    fotSal = {}
    for i in range(len(fotos)):
        r1 = np.min(fotos[i])
        r2 = np.max(fotos[i])
        fotSal[i] = np.float32(fotos[i])
        fotSal[i] = 255*(fotSal[i]-r1)/(r2-r1)
        fotSal[i] = np.uint8(fotSal[i])
        
    return fotSal

In [55]:
def trataFotos(fotos,kernelSize = 3):
    fotSal = {}
    ## Ampliación del rango
    fotSal = ampRangoSet(fotos)

    ## Suavizado
    #kernel = np.ones((kernelSize,kernelSize))/(kernelSize**2)
    n = kernelSize #tamaño del filtro
    sigma = 3 #desviación de la gaussiana
    mask = cv2.getGaussianKernel(n, sigma)*cv2.getGaussianKernel(n, sigma).T
    
    for i in range(len(fotSal)):
        fotSal[i] = cv2.filter2D(fotSal[i],-1,mask)
        
    return fotSal

In [56]:
dimFiltro = 7
fotosNegTrain = trataFotos(fotosNegTrain0,dimFiltro)
fotosPosTrain = trataFotos(fotosPosTrain0,dimFiltro)
fotosTest = trataFotos(fotosTest0,dimFiltro)

fotosNegTrain[0].shape

(134, 70)

## Extracción de Caracterísitcas

### Algoritmo HOG (Histograma de Gradientes Orientados)

El histograma de gradientes orientados (HOG) es un descriptor de características utilizado en el procesamiento de imágenes con el fin de detectar objetos. 

El algoritmo divide una imagen en porciones iguales(celdas) y cuenta las ocurrencias de orientación de gradiente en cada celda.

Vamos a usar la función hog de la librería skimage, Toda la información de esta función está en la URL:
* https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.hog
* https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html


Los parámetros de entrada son:
* **orientations**: número de orientaciones.
* **pixels_per_cell**: tamaño de la celda.
* **cells_per_block**: número de celdas por cada bloque.
* **block_norm**: método de normalización de bloques:
    * L1:
       Normalization using L1-norm.
    * L1-sqrt:
       Normalization using L1-norm, followed by square root.
    * L2:
       Normalization using L2-norm.
    * L2-Hys:
       Normalization using L2-norm, followed by limiting the
       maximum values to 0.2 (`Hys` stands for `hysteresis`) and
       renormalization using L2-norm. (default)
* **visualize**: devuelve la imagen procesada en la salida.
* **visualise**: 
* **transform_sqrt**: aplica la compresión de la ley potencial para normalizar la imagen antes del procesamiento.
* **feature_vector**: devolver los datos como un vector.


Despues de unas cuantas pruebas, definimos los parámetros de entrada:

In [5]:
orientations    = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
block_norm      = 'L2' 
visualize       = False
visualise       = None
transform_sqrt  = True
feature_vector  = True

Esta función aplica hog a todas las fotos de entrada y devuelve un array de las características de cada imagen.

la "y" es la clase a la que pertenecen los ejemplos:
* yPos = clase positiva = 1
* yNeg = clase negativa = 0

In [58]:
def featuresExtrac(imagenes,esPositivo):    
    ## Tamaño del vector
    auxDim = hog(imagenes[0],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector)
    tamHog = len(auxDim)
    
    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenesHog = np.zeros((len(imagenes),tamHog))
    
    for i in range(len(imagenes)):
        fdImagenesHog[i,:] = hog(imagenes[i],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector)
        
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenesHog,y

obtenemos la matriz de característica de las fotos Train positiva y su clase

In [59]:
fdPosTrainHog,yPos = featuresExtrac(fotosPosTrain,1)

obtenemos la matriz de característica de las fotos Train negativa y su clase

In [60]:
fdNegTrainHog,yNeg = featuresExtrac(fotosNegTrain,0)

Anexamos el conjunto positivo con el conjunto negativo para el entrenamiento.

In [61]:
fdTrainHog = np.vstack((fdPosTrainHog,fdNegTrainHog))
y = np.vstack((yPos,yNeg))


## Entrenamiento del modelo

### Support Vector Machine (SVM)

Las máquinas de vectores de soporte son son un conjunto de algoritmos de aprendizaje supervisado para resolver problemas de clasificación y regresión. Dado un conjunto de ejemplos de entrenamiento (Train) podemos etiquetar las clases y entrenar una SVM para construir un modelo que prediga la clase de una nueva muestra.

Creamos un clasificador SVM y lo entrenamos con las características de las imágenes Train(fdTrainHog) y su clase(y).

In [62]:
clfHog = svm.LinearSVC(C=0.01)
clfHog.fit(fdTrainHog[:,:],y[:,:].ravel())

LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Obtenemos la precisión de los ejemplos positivos

In [63]:
clfHog.score(fdPosTrainHog,yPos)

0.9893428063943162

Obtenemos la precisión de los ejemplos negativos

In [64]:
clfHog.score(fdNegTrainHog,yNeg)

0.99975

Como podemos ver la precisión de los ejemplos de Train es muy bueno

## Slide Window

In [26]:
def slidWindow(imagen,maskHeight = 134,maskWidth = 70,step = 10):
    arrayImagen = []
    
    for i in range(0,imagen.shape[0]-maskHeight,step):        
        for j in range(0,imagen.shape[1]-maskWidth,step):
            arrayImagen.append((i,j,imagen[i:i+maskHeight, j:j+maskWidth]))
    
    return arrayImagen

## Pyramid Gaussian

In [32]:
def estudiaFoto(im,winSize,clfHog,ratio = 2,margen = 0):#Ratio de reducción
    # List to store the detections
    hdetections = []
    detectJump = 0;
    
    for scaLev,imScaled in enumerate(pyramid_gaussian(im,downscale=ratio,multichannel=False)):
        if (imScaled.shape[0] >= winSize[0] and imScaled.shape[1] >= winSize[1]):
            for (x,y,imVentana) in slidWindow(imScaled):
                if (detectJump <= 0):
                    fdHog = hog(imVentana,orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
                    predHog = clfHog.predict(fdHog.reshape(1,-1))
                    acc = clfHog.decision_function(fdHog.reshape(1,-1))[0]
                    
                    if (predHog == 1 and acc > margen):
                        detectJump = 10
                        hdetections.append((x*(ratio**scaLev),y*(ratio**scaLev),int(winSize[1]*(ratio**scaLev)),int(winSize[0]*(ratio**scaLev)),acc ))
                detectJump -= 1;
    return hdetections

# Supresión No Maximos

### Solapamiento

In [148]:
def overlappingArea(detection_1, detection_2):
    ## Calculo de las coordenadas X Y de la detecciones
    x1_tl,y1_tl,w1,h1,_ = detection_1
    
    x1_br = x1_tl + h1
    y1_br = y1_tl + w1
    
    x2_tl,y2_tl,w2,h2,_ = detection_2
    
    x2_br = x2_tl + h2
    y2_br = y2_tl + w2
    
    
    ## Calculate the overlapping Area
    x_overlap = max(0, min(x1_br, x2_br) - max(x1_tl, x2_tl) )
    y_overlap = max(0, min(y1_br, y2_br) - max(y1_tl, y2_tl) )
    
    overlap_area = x_overlap * y_overlap
    area_1 = w1 * h1
    area_2 = w2 * h2
    total_area = area_1 + area_2 - overlap_area
    
    return overlap_area / float(total_area)

In [132]:
def SNoM(detections, umbral  = 0):
    GoodDetec = []
    
    ## Ordenamos las deteciones por coefiente de precision
    detections = sorted(detections, key=lambda detections: detections[4],reverse=True)
    
    GoodDetec.append(detections[0])
    del detections[0]
    
    for i,oldDetec in enumerate(detections):
        for gDetec in GoodDetec:
            if overlappingArea(oldDetec, gDetec) > umbral:
                del detections[i]
                break
        else:
            GoodDetec.append(oldDetec)
            del detections[i]

    return GoodDetec

# Prueba del Modelo

In [92]:
def muestraResultado(detections,im0):
    color = (255)
    im = im0.copy()
    for (x, y,w, h,_) in detections:
            # Draw the detections
            cv2.rectangle(im, (y, x), (y+w, x+h), color, thickness=1)
    showImg(im,"Deteciones")

In [65]:
%%time
humanDetec = estudiaFoto(fotosTest[0],winSize,clfHog,ratio=2)

Wall time: 14.5 s


In [66]:
len(humanDetec)

16

In [118]:
muestraResultado(humanDetec,fotosTest[0])

In [149]:
muestraResultado(SNoM(humanDetec,umbral = 0.1),fotosTest[0])

0.25
0.25
0.0
0.8611111111111112
0.0
0.0
0.7402597402597403
0.0
0.0
0.0
0.0
0.0
0.25


#### Prueba solo Una imagen

In [ ]:
%%time
listOk = []
partImagen = slidWindow(fotosTest[7])
lim = 0.5
for i in range(len(partImagen)):
    pruebaHog = hog(partImagen[i][2],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
    precis = clfHog.decision_function(pruebaHog.reshape(1,-1))[0]
    if (1 == clfHog.predict(pruebaHog.reshape(1,-1)) and precis > lim):
        listOk.append(i);
        showImg(partImagen[i][2])

In [ ]:
print(len(partImagen))
print(len(listOk))
print(listOk)

In [ ]:
cv2.imshow('img ', fotosTest[0])
cv2.imshow('trozo test ', partImagen[3520][2])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
pruebaHog = hog(partImagen[3520][2],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
porce = clfHog.decision_function(pruebaHog.reshape(1,-1))[0]
print(porce)